In [1]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from tabulate import tabulate
from tqdm import tqdm
import numpy as np
import networkx as nx

from debugger.edit_info import EditAction
from debugger.debugger import Debugger3
from debugger.tree_node import NodeRef
from evaluator import Evaluator

In [2]:
rev = Evaluator("84faa1a2fa")
editor = rev.editor
ratios = rev.get_trace_graph_ratios()
report = rev.build_report()
mi = rev.get_trace_info()
trace_graph = mi.get_trace_graph()

[DEBUG] [edit] proof path: dbg/84faa1a2fa/proofs/rename.2567182715268410107.proof 
[DEBUG] [edit] trace path: dbg/84faa1a2fa/traces/rename.3991909809416166522 


f43f5a217ee7576ffee8847579c72a79.pickle
[WARN] [differ] qid constructor_accessor_axiom not found in dbg/84faa1a2fa/orig.smt2 
[INFO] [graph] found dbg/84faa1a2fa/graphs/rename.3991909809416166522.txt 
[INFO] [stats] found dbg/84faa1a2fa/stats/rename.3991909809416166522.txt 


In [ ]:
report.print_insted()

In [ ]:
report.print_tested()

In [4]:
roots = dict()
singleton_scores = dict()

for root in list(editor.list_qnames(root_only=True)):
    if root not in ratios:
        continue

    actions = editor.get_quant_actions(root)

    if (
        actions == {EditAction.SKOLEMIZE}
        or actions == {EditAction.NONE}
        or actions == {EditAction.ERROR}
    ):
        continue

    roots[root] = set(editor.trace_stats.get_group_stat(root).keys())
    singleton_scores[root] = trace_graph.aggregate_scores(ratios[root])

singleton_ranked = [
    (k, v)
    for k, v in sorted(singleton_scores.items(), key=lambda item: item[1], reverse=True)
]

In [14]:
ranked = [x[0] for x in singleton_ranked]
TOP_N = 50
# print(len(singleton_ranked))

In [15]:
from debugger.informed_editor import choose_action

scores = dict()

for i, root1 in enumerate(tqdm(ranked[:TOP_N])):
    ratio1 = ratios[root1]
    group1 = roots[root1]
    cur_best, selected = 0, None

    for root2 in ranked[TOP_N+1:2*TOP_N+1]:
        ratio2 = ratios[root2]  
        group2 = roots[root2]
        merged = {}

        for k in ratio1.keys():
            merged[k] = max(ratio1[k], ratio2.get(k, 0))
        for k in ratio2.keys():
            merged[k] = max(ratio2[k], ratio1.get(k, 0))
        for k in list(merged.keys()):
            if merged[k] == 0:
                merged.pop(k)
        starts = group1 | group2

        res = trace_graph.compute_sub_ratios(starts=starts, bootstrap=merged)
        score = trace_graph.aggregate_scores(res)
        if score > cur_best:
            cur_best = score
            selected = root2

    scores[(root1, selected)] = cur_best

scores = [(k, v) for k, v in scores.items() if v > 0]
sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
sorted_scores = sorted_scores[:TOP_N]



100%|██████████| 50/50 [00:10<00:00,  4.76it/s]


In [16]:
for (qname1, qname2), score in sorted_scores:
    action1 = choose_action(editor.get_quant_actions(qname1))
    action2 = choose_action(editor.get_quant_actions(qname2))
    # edit = {qname1: action1, qname2: action2}
    print(f"{qname1} {qname2} {action1.value} {action2.value} {score}")

mariposa_qid_390 mariposa_qid_405 erase erase 17123.05183218475
mariposa_qid_1716 mariposa_qid_405 inst_replace erase 14591.903250302435
mariposa_qid_1698 mariposa_qid_405 erase erase 10800.705481629047
mariposa_qid_363 mariposa_qid_405 inst_replace erase 10037.868985775885
mariposa_qid_1704 mariposa_qid_1570 inst_replace erase 9521.501002282705
mariposa_qid_388 mariposa_qid_405 inst_replace erase 9362.098538772922
mariposa_qid_2814 mariposa_qid_405 erase erase 7465.443827593559
mariposa_qid_1683 mariposa_qid_405 erase erase 3142.7324074074077
mariposa_qid_1695 mariposa_qid_405 erase erase 3107.445879342258
mariposa_qid_56 mariposa_qid_405 inst_replace erase 2548.5360064084007
mariposa_qid_1709 mariposa_qid_405 inst_replace erase 2342.6159376970136
mariposa_qid_127 mariposa_qid_405 erase erase 1930.6599998697857
mariposa_qid_332 mariposa_qid_405 inst_replace erase 1571.8144911851757
mariposa_qid_89 mariposa_qid_405 inst_replace erase 1354.5794956067905
mariposa_qid_1706 mariposa_qid_40

In [17]:
editor.edit_by_qname("mariposa_qid_390")
editor.edit_by_qname("mariposa_qid_405")
editor.save("test.smt2")

[INFO] [edit] [erase] mariposa_qid_390 
[INFO] [edit] [erase] mariposa_qid_405 
[INFO] [edit] written to test.smt2 
